<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E5%9B%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
# 步驟 1: 安裝依賴並載入 API Key
!pip install -q gspread pandas requests beautifulsoup4 jieba scikit-learn gradio

import os
from google.colab import userdata

# 請將您的 API Key 儲存為 Colab Secret，名稱為 GEMINI_API_KEY
# 這裡將 Secret 載入到環境變數中
os.environ["gemini"] = userdata.get("gemini")

# 檢查是否成功載入 (只會顯示前幾位字符，是安全的)
if os.environ["gemini"]:
    print(f"✅ Gemini API Key 載入成功！(Prefix: {os.environ['gemini'][:5]}...)")
else:
    print("❌ 警告：請確認您已在 Colab Secrets 中設置 'GEMINI_API_KEY'！")

# 執行 Google Sheet 認證 (使用您自己的 Google 帳號)
from google.colab import auth
auth.authenticate_user()

print("✅ Google Colab 認證成功！")

✅ Gemini API Key 載入成功！(Prefix: AIzaS...)
✅ Google Colab 認證成功！


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import json
import time
import os # <-- 確保導入 os
import gradio as gr

# Google Sheet 相關套件
import gspread
from google.auth import default

# --- 配置區塊 ---
PTT_BOARD = "KoreaStar"
BASE_URL = f"https://www.ptt.cc/bbs/{PTT_BOARD}/"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Gemini API 配置：從環境變量中獲取 API Key
# 必須先在步驟 1 將 Secret 載入到 os.environ["GEMINI_API_KEY"]
API_KEY = os.environ.get("GEMINI_API_KEY", "")

# 檢查 API Key 是否存在，如果不存在，使用一個錯誤提示符號
display_key = API_KEY if API_KEY else "INVALID_OR_MISSING_KEY"
print(f"Gemini API URL 使用的 Key 前綴: {display_key[:5]}...")

GEMINI_API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key={API_KEY}"


# --- Google Sheet 配置 (使用您提供的 URL 和工作表名稱) ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=579218704#gid=579218704"
WORKSHEET_NAME = "工作表4"

# 停用詞列表
STOPWORDS = set(['的', '了', '是', '在', '我', '你', '他', '她', '之', '一個', '和', '討論', '分享', '報導', '新聞', '娛樂', '記者', '綜合', '發信', '站內', '標題', '時間', '來源', 'ptt', '發文', '作者', '沒有', '最新', '真的', '什麼', '大家', '就是', '這個', '那個', '可以'])


# --- Google Sheet 連接函數 (使用 Colab 認證) ---
def gsheet_connect_colab():
    """使用 Colab 認證連接 Google Sheet"""
    try:
        # 使用 Colab 認證獲取憑證
        creds, _ = default()
        gc = gspread.authorize(creds)

        # 開啟試算表
        sh = gc.open_by_url(SHEET_URL)

        # 嘗試獲取工作表，如果不存在則創建
        try:
            worksheet = sh.worksheet(WORKSHEET_NAME)
        except gspread.WorksheetNotFound:
            # 創建新的工作表，並設定行數/列數
            worksheet = sh.add_worksheet(title=WORKSHEET_NAME, rows="1000", cols="5")

        return worksheet, "GSHEET_COLAB"

    except Exception as e:
        print(f"Google Sheet 連接或認證失敗: {e}")
        return None, "ERROR"


# --- 1. 爬蟲核心函數 ---

def get_latest_index(board_url):
    """獲取最新文章頁面的上一頁 (即第一頁的頁碼)"""
    try:
        response = requests.get(board_url, headers=HEADERS, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        paging_div = soup.find('div', class_='btn-group btn-group-paging')
        if paging_div:
            prev_button = paging_div.find_all('a')[1]
            if 'href' in prev_button.attrs:
                match = re.search(r'index(\d+)\.html', prev_button['href'])
                if match:
                    # 返回當前最新頁的頁碼
                    return int(match.group(1)) + 1
        return 1
    except Exception:
        return 1

def get_article_content(article_href):
    """獲取單篇文章的內文"""
    if article_href.startswith('N/A'):
        return "文章已被刪除或不可存取。"

    try:
        response = requests.get(article_href, headers=HEADERS, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        main_content = soup.find(id="main-content")

        if not main_content:
            return "無法找到文章內容。"

        # 清理 Meta 資訊、推文和頁底資訊
        for tag_class in ['article-metaline', 'article-metaline-right', 'push', 'f2', 'f6']:
             for tag in main_content.find_all('div', class_=tag_class):
                tag.extract()

        content = main_content.text.strip()
        content = re.sub(r'(※ 發信站: .*|※ 文章網址: .*|作者: .*|\n)', '', content).strip()
        return content

    except Exception:
        return "無法抓取內文。"


def crawl_ptt_koreastar(pages_to_fetch):
    """主爬蟲函數：爬取多頁文章列表並抓取內文，並寫入 Google Sheet"""

    start_index = get_latest_index(BASE_URL)
    stop_index = max(1, start_index - pages_to_fetch)
    articles_data = []

    for index in range(start_index, stop_index, -1):
        url = f"{BASE_URL}index{index}.html"

        try:
            response = requests.get(url, headers=HEADERS, timeout=5)
            if response.status_code != 200:
                 continue

            soup = BeautifulSoup(response.text, 'html.parser')
            article_list = soup.find_all('div', class_='r-ent')

            for article in article_list:
                title_tag = article.find('div', class_='title').find('a')

                if title_tag:
                    title = title_tag.text.strip()
                    href = "https://www.ptt.cc" + title_tag['href']
                    content = get_article_content(href)
                    author = article.find('div', class_='author').text.strip()
                    date = article.find('div', class_='date').text.strip()

                    articles_data.append({
                        'title': title,
                        'date': date,
                        'author': author,
                        'href': href,
                        'content': content
                    })

            time.sleep(0.5)

        except Exception:
            continue

    df = pd.DataFrame(articles_data)
    df = df[df['content'].str.strip() != '文章已被刪除或不可存取。'].reset_index(drop=True)

    if df.empty:
        return df, "❌ 爬取完成，但未找到有效文章。"

    # 步驟 3: 寫入 Google Sheet
    worksheet, connection_type = gsheet_connect_colab()

    if connection_type == "GSHEET_COLAB" and worksheet:
        try:
            # 清除現有內容 (保留標頭)
            worksheet.clear()

            # 準備數據，包含標頭
            cols_order = ['title', 'date', 'author', 'href', 'content']
            df = df[cols_order]
            data_to_write = [df.columns.values.tolist()] + df.values.tolist()

            # 寫入數據
            worksheet.update(data_to_write, value_input_option='USER_ENTERED')

            status_msg = f"✅ 成功爬取 {len(df)} 篇文章並**寫入 Google Sheet: {WORKSHEET_NAME}**"

        except Exception as e:
            status_msg = f"⚠️ Google Sheet 寫入失敗 ({e})。請確認您是否已執行 `auth.authenticate_user()` 且擁有編輯權限。"

    else:
        status_msg = "❌ Google Sheet 連接失敗，無法寫入。請確認 Colab 認證是否成功。"

    return df, status_msg


# --- 2. 數據讀取與文本分析核心函數 ---

def analyze_keywords(df, top_n=10):
    """執行 TF-IDF 關鍵字統計"""
    if df.empty:
        return pd.DataFrame(), "錯誤: 資料集為空，無法進行分析。"

    documents = []

    for _, row in df.iterrows():
        # 標題權重 * 3
        text = row['title'] * 3 + " " + row['content']
        cleaned_text = re.sub(r'[^\u4e00-\u9fff\w]', ' ', text)
        words = jieba.lcut(cleaned_text, cut_all=False)

        filtered_words = [
            word.strip().lower()
            for word in words
            if word.strip() and len(word.strip()) > 1 and word.strip().lower() not in STOPWORDS and not word.isdigit()
        ]
        documents.append(" ".join(filtered_words))

    if not any(documents):
         return pd.DataFrame(), "錯誤: 數據清理後無有效詞彙進行 TF-IDF 分析。"

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)

    feature_names = vectorizer.get_feature_names_out()
    tfidf_array = tfidf_matrix.toarray()

    # 計算詞彙的 TF-IDF 平均權重
    avg_tfidf_scores = defaultdict(float)
    num_documents = len(documents)

    for doc_weights in tfidf_array:
        for i, weight in enumerate(doc_weights):
            avg_tfidf_scores[feature_names[i]] += weight

    for word in avg_tfidf_scores:
        avg_tfidf_scores[word] /= num_documents

    sorted_avg_tfidf = sorted(avg_tfidf_scores.items(), key=lambda item: item[1], reverse=True)

    top_n_keywords = sorted_avg_tfidf[:top_n]

    keyword_df = pd.DataFrame(top_n_keywords, columns=['關鍵字 (Top N 熱詞)', 'TF-IDF 平均權重'])
    keyword_df['TF-IDF 平均權重'] = keyword_df['TF-IDF 平均權重'].round(4)

    return keyword_df, f"✅ TF-IDF 關鍵字統計成功，輸出 Top {top_n} 熱詞。"


# --- 3. Gemini API 串接核心函數 ---

def generate_summary(keyword_df, df_articles, user_question):
    """串接 Gemini API 生成洞察摘要與結論"""
    if keyword_df.empty or df_articles.empty:
        return "無法生成摘要。請先執行爬蟲與分析步驟。", ""

    # 如果 API Key 遺失，直接返回錯誤
    if not API_KEY:
        return "❌ 錯誤：Gemini API Key 尚未設置或無效。", ""


    top_keywords = ", ".join(keyword_df['關鍵字 (Top N 熱詞)'].tolist())
    article_titles = "\n- " + "\n- ".join(df_articles['title'].head(30).tolist())

    system_prompt = (
        "你是一位專業的韓國娛樂圈趨勢分析師。你的任務是基於提供的 PTT 韓星版數據，"
        "分析最近的討論熱點、趨勢、以及使用者最關心的問題（如果提供的話）。"
        "回答必須是繁體中文，且格式需嚴格遵循要求。"
    )

    user_prompt = (
        "請根據以下數據生成分析結果：\n\n"
        "1. **熱門關鍵字 (TF-IDF Top 10):**\n"
        f"{top_keywords}\n\n"
        "2. **最新熱門文章標題 (最近30篇):**\n"
        f"{article_titles}\n\n"
        "3. **使用者感興趣的問題:**\n"
        f"'{user_question}'\n\n"
        "分析要求:\n"
        "A. 生成 5 句**具體的**洞察摘要 (Insights)，每句單獨以星號開頭，單獨成行。\n"
        "B. 生成一段約 120 字的結論，總結韓國娛樂圈近期在紅什麼。"
    )

    try:
        response = requests.post(
            GEMINI_API_URL,
            headers={'Content-Type': 'application/json'},
            data=json.dumps({
                "contents": [{"parts": [{"text": user_prompt}]}],
                "systemInstruction": {"parts": [{"text": system_prompt}]},
            }),
            timeout=60
        )

        response.raise_for_status()
        result = response.json()

        generated_text = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'API 返回空結果。')

        lines = generated_text.split('\n')
        insights = []
        conclusion_parts = []

        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('*') or re.match(r'^[\d]\. ', stripped_line):
                insights.append(stripped_line.lstrip('* ').lstrip('12345. '))
            elif stripped_line:
                conclusion_parts.append(stripped_line)

        insight_output = "\n* " + "\n* ".join(insights[:5])
        conclusion_output = " ".join(conclusion_parts).strip()

        if not insight_output.strip() or not conclusion_output.strip():
            return "* (解析失敗，原始輸出如下:)\n" + generated_text, "(解析失敗，請見摘要區)"

        return insight_output, conclusion_output

    except requests.exceptions.HTTPError as e:
        error_msg = f"❌ Gemini API 呼叫失敗: {e}。請檢查 API Key 是否有效或是否有足夠的餘額。"
        return error_msg, ""
    except requests.exceptions.RequestException as e:
        error_msg = f"❌ Gemini API 呼叫失敗: {e}"
        return error_msg, ""
    except Exception as e:
        return f"處理 Gemini 響應時發生錯誤: {e}", ""


# --- 4. Gradio 介面流程函數 ---

def full_analysis_workflow(pages_to_fetch, user_question, top_n=10):
    """一鍵執行爬蟲、分析、總結的完整流程"""
    if pages_to_fetch <= 0:
        return (pd.DataFrame(), "❌ 錯誤: 爬取頁數必須大於 0。", "分析失敗", "分析失敗", "分析失敗")

    # 步驟 1: 執行爬蟲並寫入 Google Sheet
    df_articles, crawl_status = crawl_ptt_koreastar(pages_to_fetch)

    if df_articles.empty:
        return (pd.DataFrame(), crawl_status, "分析失敗", "分析失敗", "分析失敗")

    # 步驟 2: 執行 TF-IDF 關鍵字統計 (使用爬取到的 DataFrame)
    keyword_df, analyze_status = analyze_keywords(df_articles, top_n)

    # 步驟 3: 串接 Gemini API 生成摘要與結論
    insight_output, conclusion_output = generate_summary(keyword_df, df_articles, user_question)

    keyword_markdown = keyword_df.to_markdown(index=False)

    final_status = f"{crawl_status}\n{analyze_status}"

    return (df_articles, final_status, keyword_markdown, insight_output, conclusion_output)

def ask_question_workflow(question):
    """問答機器人流程：直接使用 Gemini API 回答問題"""
    if not question:
        return "請輸入您的問題。"

    # 如果 API Key 遺失，直接返回錯誤
    if not API_KEY:
        return "❌ 錯誤：Gemini API Key 尚未設置或無效。"

    system_prompt = (
        "你是一位專門回答韓國娛樂圈、K-Pop 相關問題的專家。 "
        "請以流暢的繁體中文，親切地回答使用者的問題。 "
        "如果答案需要最新資訊，請使用搜尋工具來輔助回答。"
    )

    try:
        response = requests.post(
            GEMINI_API_URL,
            headers={'Content-Type': 'application/json'},
            data=json.dumps({
                "contents": [{"parts": [{"text": question}]}],
                "systemInstruction": {"parts": [{"text": system_prompt}]},
                "tools": [{"google_search": {}}]
            }),
            timeout=60
        )

        response.raise_for_status()
        result = response.json()

        generated_text = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'API 返回空結果。')

        sources_text = ""
        grounding_metadata = result.get('candidates', [{}])[0].get('groundingMetadata')
        if grounding_metadata and grounding_metadata.get('groundingAttributions'):
            sources = []
            for attr in grounding_metadata['groundingAttributions']:
                web = attr.get('web')
                if web and web.get('title') and web.get('uri'):
                    sources.append(f"[{web['title']}]({web['uri']})")

            if sources:
                sources_text = "\n\n---\n**參考來源:**\n" + "\n".join(sources[:5])

        return generated_text + sources_text

    except requests.exceptions.HTTPError as e:
        return f"❌ Gemini API 呼叫失敗: {e}。請檢查 API Key 是否有效或是否有足夠的餘額。"
    except requests.exceptions.RequestException as e:
        return f"❌ Gemini API 呼叫失敗: {e}"
    except Exception as e:
        return f"處理 Gemini 響應時發生錯誤: {e}"


# --- 5. Gradio 介面構建 ---

with gr.Blocks(title="PTT 韓星趨勢分析系統", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # PTT 韓星版 (KoreaStar) 趨勢分析系統 🇰🇷✨
        **運行步驟：**
        1. **請先執行步驟 1 程式碼區塊**，確保 Gemini API Key 與 Google Sheet 認證成功。
        2. 在下方設定爬取頁數與問題，然後點擊執行。
        3. 爬取結果將寫入您的 Google Sheet **工作表4**。
        """
    )

    with gr.Tab("韓星趨勢分析 (一鍵執行)"):
        gr.Markdown("## 📈 趨勢分析與洞察生成")

        with gr.Row():
            pages_slider = gr.Slider(
                minimum=1, maximum=30, value=10, step=1, label="要爬取的 PTT 頁數",
                info="爬取越多頁，分析結果越全面，但耗時越久。建議從 10 頁開始。",
                interactive=True
            )
            top_n_slider = gr.Slider(
                minimum=5, maximum=20, value=10, step=1, label="Top N 熱詞",
                info="TF-IDF 統計要輸出的熱詞數量。",
                interactive=True
            )

        user_input = gr.Textbox(
            label="您目前最關心的韓國娛樂圈問題/團體是什麼？",
            placeholder="例如：最近 NewJeans 的回歸討論度如何？ (此問題會提供給 AI 參考)",
            lines=2
        )

        run_button = gr.Button("🚀 一鍵執行趨勢分析與 Gemini 總結", variant="primary")

        status_text = gr.Markdown("--- 執行狀態 ---", elem_id="status_output")

        gr.Markdown("## 📋 分析結果")

        keyword_output = gr.Markdown(
            label="TF-IDF 熱詞統計",
            value="等待執行..."
        )

        data_preview = gr.DataFrame(
            label="爬蟲數據預覽 (已寫入 Google Sheet '工作表4' 的內容)",
            wrap=True
        )

        gr.Markdown("## 🧠 Gemini AI 洞察")

        insight_output = gr.Markdown(
            label="5 句洞察摘要",
            value="* 等待執行..."
        )

        conclusion_output = gr.Markdown(
            label="120 字結論 (韓國娛樂圈近期在紅什麼？)",
            value="等待執行..."
        )

        run_button.click(
            fn=full_analysis_workflow,
            inputs=[pages_slider, user_input, top_n_slider],
            outputs=[data_preview, status_text, keyword_output, insight_output, conclusion_output]
        )

    with gr.Tab("AI 問答機器人"):
        gr.Markdown("## 🤖 韓國娛樂圈問答機器人")

        question_input = gr.Textbox(
            label="請輸入您對韓國娛樂圈好奇的問題：",
            placeholder="例如：最近有哪些 K-POP 團體發行了新歌？"
        )
        ask_button = gr.Button("提問 (串接 Gemini API 搜尋最新資訊)", variant="secondary")

        answer_output = gr.Markdown(label="AI 回答", value="等待提問...")

        ask_button.click(
            fn=ask_question_workflow,
            inputs=[question_input],
            outputs=[answer_output]
        )

# 啟動 Gradio 應用
if __name__ == "__main__":
    jieba.initialize()
    demo.launch(debug=True)

Gemini API URL 使用的 Key 前綴: INVAL...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6adc33478603f2dd3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
